In [66]:
# Constants
DF_PATH_PKL = "../data/raw/1_collected_raw_data_df.pkl"
DF_PATH_CSV = "../data/raw/1_collected_raw_data_df.csv"

FIG_DIR = "../reports/figures"

REPLACED_DICT = {
    'Founded' : {'Nonprofit Organization' : -1, 'Government' : -1, 'Company - Private' : -1,
                 'Government' : -1, 'Company - Public': -1}
}

In [45]:
# Load packages
import pandas as pd
import numpy as np

In [46]:
# Read data and print shape
raw_df = pd.read_pickle(DF_PATH_PKL)
display(raw_df.shape)
# display(raw_df.head(2))
display(raw_df.columns)
display(raw_df.info())

(1000, 14)

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Job Title          1000 non-null   object
 1   Salary Estimate    1000 non-null   object
 2   Job Description    1000 non-null   object
 3   Rating             1000 non-null   object
 4   Company Name       1000 non-null   object
 5   Location           1000 non-null   object
 6   Headquarters       1000 non-null   object
 7   Size               1000 non-null   object
 8   Founded            1000 non-null   object
 9   Type of ownership  1000 non-null   object
 10  Industry           1000 non-null   object
 11  Sector             1000 non-null   object
 12  Revenue            1000 non-null   object
 13  Competitors        1000 non-null   int64 
dtypes: int64(1), object(13)
memory usage: 109.5+ KB


None

In [55]:
# Invesigate the questionable objects columns
questionable_cols = ['Salary Estimate','Rating','Founded']

for col in questionable_cols:
    print(col)
    print(raw_df[col].unique().tolist())
    print('--------------------------------------------')
    print()

Salary Estimate
['$87K - $121K (Glassdoor est.)', '$70K - $108K (Glassdoor est.)', 'Employer Provided Salary:$67K - $120K', '$114K - $164K (Glassdoor est.)', 'Employer Provided Salary:$65K - $80K', '$99K - $145K (Glassdoor est.)', 'Employer Provided Salary:$50.00 - $55.00 Per Hour', '$84K - $129K (Glassdoor est.)', 'Employer Provided Salary:$105K - $135K', '$81K - $113K (Glassdoor est.)', '$97K - $138K (Glassdoor est.)', '$112K - $189K (Glassdoor est.)', '$76K - $110K (Glassdoor est.)', 'Employer Provided Salary:$100K - $225K', 'Employer Provided Salary:$98K - $116K', '$105K - $170K (Glassdoor est.)', 'Employer Provided Salary:$114K - $131K', '$114K - $172K (Glassdoor est.)', 'Employer Provided Salary:$150K - $160K', 'Employer Provided Salary:$78K - $150K', '$84K - $123K (Glassdoor est.)', '$91K - $122K (Glassdoor est.)', '$79K - $125K (Glassdoor est.)', 'Employer Provided Salary:$115K - $144K', '$88K - $142K (Glassdoor est.)', '$80K - $122K (Glassdoor est.)', 'Employer Provided Salary

In [48]:
# Tasks on data to do
##### slary parsing
##### Company name text only
##### state field
##### age of compny
##### parsing of job description (python, etc)

#####################################################
#[1] slary parsing
# raw_df[raw_df['Salary Estimate'] == -1]                               # filter to view
raw_df = raw_df[raw_df['Salary Estimate'] != -1]
salary = raw_df['Salary Estimate'].apply(lambda x: x.split('(')[0])
minus_kd = salary.apply(lambda x: x.replace('K','').replace('$',''))

# raw_df.loc[raw_df['Salary Estimate'].str.contains('Per Hour')]        # filter to view
raw_df["hourly"] = raw_df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
raw_df["employer_provided"] = raw_df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary:' in x.lower() else 0)

min_hr = minus_kd.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary:',''))

# on 'min_salary' : because there can not be min salary = 0, so if there is only one value on data , it will be the max and min slary
raw_df['min_salary'] = min_hr.apply(lambda x: x.split('-')[0] if len(x.split('-'))==1 else x.split('-')[0] ).astype(np.float32).astype(np.int64)
raw_df['max_salary'] = min_hr.apply(lambda x: x.split('-')[0] if len(x.split('-'))==1 else x.split('-')[1] ).astype(np.float32).astype(np.int64)
raw_df['avg_salary'] = (raw_df['min_salary'] + raw_df['max_salary'])/2

In [49]:
duplicate = raw_df[raw_df.duplicated()]
duplicate.shape

(546, 19)

In [50]:
#[2] Company name text only
# Converting 'Rating' to int 
raw_df['Rating'] = raw_df['Rating'].astype(np.float32).astype(np.int64)
display(raw_df['Rating'].dtype)
# raw_df['company_txt'] = raw_df.apply(lambda x: x['Company Name'] if x['Rating'] <0 else x['Company Name'][:-3], axis=1)
raw_df['company_txt'] = raw_df['Company Name'].str.replace('[0-9]','').str.replace('.','')

dtype('int64')

/Users/ahmadsamir/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/Users/ahmadsamir/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [51]:
# [3] state field
raw_df['job_state'] = raw_df['Location'].apply(lambda x: x.split(',')[0] if len(x.split(',')) == 1 else x.split(',')[0])
raw_df['job_state'].value_counts()
# raw_df['some_state'] = raw_df.apply(lambda x: 1 if x['Location'] == x['Headquarters'] else 0, axis=1)

Remote                404
Mountain View         246
O'Fallon               58
New York               30
Fort Meade             28
Cupertino              27
Annapolis Junction     25
Washington             17
Name: job_state, dtype: int64

In [70]:
# [4] age of company
# replace non integer values with -1
for col, replacement in REPLACED_DICT.items() :
    raw_df[col] = raw_df[col].replace(replacement).astype(np.int64)
raw_df['age'] = raw_df['Founded'].apply(lambda x: x if x < 1 else 2022 - x)